In [ ]:
import os
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from urllib.request import urlopen
import json
    
# Kaleido package version 0.1.0.post1 [pip] or 0.1.0 [conda] is needed to save figures.

In [ ]:
# Load Result Data
loc_R = 'Data_Output/Results.csv'
df_result = pd.read_csv(loc_R)

# Parse Result Data
ns_data = df_result.iloc[:, 0]
yc_data = df_result.iloc[:, 1]
ys_data = df_result.iloc[:, 2]     
ep_data = df_result.iloc[:, 3]

# Load County Geogrphy data
loc_C = 'Data_Input/county_data/County_List.csv'
df_C = pd.read_csv(loc_C)
# Parse County Geogrphy data
fips = df_C["FIPS Code"]

# National Data for all Counties
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

year = pd.Series(range(1997, 2020))

# Define Dropdown Widget
output_dropdown = widgets.Dropdown(
    options=[('Nitrogen Surplus', 1), ('Corn Yield', 2), ('Soybean Yield', 3), ('Enthanol Production', 4)],
    description='Output:',
    disabled = False)

# Define Slider Widget
year_slider = widgets.IntSlider(
    min = 1997,
    max = 2019,
    step = 1,
    continuous_update=False, # Updates widget only on mouse relase events to improve performance.
    description='Year:',
    disabled = False)

In [ ]:
def plot_function(output_no, year):
    
    year = year - 1997
    
    if output_no == 1:
        values = ns_data.iloc[range(year, len(ns_data), 23)]
        lower_bound = min(ns_data)
        upper_bound = max(ns_data)
        var_name = "Nitrogen Surplus"
        unit_name = "[kg/ha]"
    elif output_no == 2:
        values = yc_data.iloc[range(year, len(yc_data), 23)]
        lower_bound = min(yc_data)
        upper_bound = max(yc_data)
        var_name = "Corn Yield"
        unit_name = "[bu/acre]"
    elif output_no == 3:
        values = ys_data.iloc[range(year, len(ys_data), 23)]
        lower_bound = min(ys_data)
        upper_bound = max(ys_data)
        var_name = "Soybean Yield"
        unit_name = "[bu/acre]"
    elif output_no == 4:
        values = ep_data.iloc[range(year, len(ep_data), 23)] / 1e6 # Convert to Millions of Gals
        lower_bound = min(ep_data) / 1e6
        upper_bound = max(ep_data) / 1e6
        var_name = "Ethanol Production"
        unit_name = "[Mgal]"
    
    df = pd.concat([fips, values], axis = 1)
    
    fig = px.choropleth(df, geojson = counties, locations = fips, color = values,
                        color_continuous_scale = "Turbo",
                        # color_continuous_scale = "Viridis",
                        range_color = (lower_bound, upper_bound),
                        scope = "usa",
                        title = '{} in Iowa ({})'.format(var_name, str(year + 1997)))

    fig.update_layout(margin = {"r":0,"t":10,"l":0,"b":0},
                      title_x = 0.45,
                      title_y = 0.97,
                      coloraxis_colorbar = dict(
                          title = unit_name,
                          y = 0.5,
                          x = 1))
    
    fig.update_geos(fitbounds = "locations") 
    return fig

### Interative Widgets

In [ ]:
# Interactive Visualization
widgets.interact(plot_function, year = year_slider, output_no = output_dropdown)

### Save Figures

Note: Individually saves the plots for each year and output variable as png files. Can take several minutes to execute.

In [ ]:
# Creates Directory
if not os.path.exists("Figures"):
    os.mkdir("Figures")
if not os.path.exists("Figures/Nitrogen Surplus"):
    os.mkdir("Figures/Nitrogen Surplus")
if not os.path.exists("Figures/Corn Yield"):
    os.mkdir("Figures/Corn Yield")
if not os.path.exists("Figures/Soybean Yield"):
    os.mkdir("Figures/Soybean Yield")
if not os.path.exists("Figures/Ethanol Production"):
    os.mkdir("Figures/Ethanol Production")

In [ ]:
# Code to Save All Figures
for output_no in range(1, 5):
    for year in range(1997, 2020):
        year = year - 1997
        bins = 20j

        if output_no == 1:
            values = ns_data.iloc[range(year, len(ns_data), 23)]
            lower_bound = min(ns_data)
            upper_bound = max(ns_data)
            var_name = "Nitrogen Surplus"
            unit_name = "[kg/ha]"
        elif output_no == 2:
            values = yc_data.iloc[range(year, len(yc_data), 23)]
            lower_bound = min(yc_data)
            upper_bound = max(yc_data)
            var_name = "Corn Yield"
            unit_name = "[bu/acre]"
        elif output_no == 3:
            values = ys_data.iloc[range(year, len(ys_data), 23)]
            lower_bound = min(ys_data)
            upper_bound = max(ys_data)
            var_name = "Soybean Yield"
            unit_name = "[bu/acre]"
        elif output_no == 4:
            values = ep_data.iloc[range(year, len(ep_data), 23)] / 1e6 # Convert to Millions of Gals
            lower_bound = min(ep_data) / 1e6
            upper_bound = max(ep_data) / 1e6
            var_name = "Ethanol Production"
            unit_name = "[Mgal]"
            
        df = pd.concat([fips, values], axis = 1)

        fig = px.choropleth(df, geojson = counties, locations = fips, color = values,
                            color_continuous_scale = "Turbo",
                            # color_continuous_scale = "Viridis",
                            range_color = (lower_bound, upper_bound),
                            width = 900,
                            height = 450,
                            scope = "usa",
                            title = '{} in Iowa ({})'.format(var_name, str(year + 1997)))

        fig.update_layout(margin = {"r":0,"t":25,"l":5,"b":5},
                          title_x = 0.45,
                          title_y = 0.98,
                          coloraxis_colorbar = dict(title = unit_name))

        fig.update_geos(fitbounds = "locations") 
        
        fig_name = "Figures/{}/{} - {}.png".format(var_name, var_name, str(year + 1997))
        fig.write_image(fig_name, scale = 5)
        
print("\nAll figures have been saved!!!\n")